# Install [Kubeapps](https://kubeapps.com/)

### Create service accounts

In [ ]:
source ../config.sh

kubectl config use-context ${ALPHA_CLUSTER}

echo
kubectl create namespace kubeapps
kubectl create -n kubeapps serviceaccount kubeapps-operator
kubectl create clusterrolebinding kubeapps-operator --clusterrole=cluster-admin --serviceaccount=kubeapps:kubeapps-operator

echo
echo 'Token for kubeapps on Alpha cluster:'
kubectl get -n kubeapps secret $(kubectl get -n kubeapps serviceaccount kubeapps-operator -o jsonpath='{.secrets[].name}') -o go-template='{{.data.token | base64decode}}' && echo

echo
kubectl config use-context ${BRAVO_CLUSTER}

echo
kubectl create namespace kubeapps
kubectl create -n kubeapps serviceaccount kubeapps-operator
kubectl create clusterrolebinding kubeapps-operator --clusterrole=cluster-admin --serviceaccount=kubeapps:kubeapps-operator

echo
echo 'Token for kubeapps on Bravo cluster:'
kubectl get -n kubeapps secret $(kubectl get -n kubeapps serviceaccount kubeapps-operator -o jsonpath='{.secrets[].name}') -o go-template='{{.data.token | base64decode}}' && echo

### Configure Argo CD to deploy Kubeapps

In [ ]:
source ../config.sh

ALPHA_CLUSTER_URL=`tmc cluster provisionedcluster kubeconfig get ${ALPHA_CLUSTER} | yq r - 'clusters[0].cluster.server'`
BRAVO_CLUSTER_URL=`tmc cluster provisionedcluster kubeconfig get ${BRAVO_CLUSTER} | yq r - 'clusters[0].cluster.server'`

kubectl config use-context ${CICD_CLUSTER}

echo
echo 'Configure the Kubeapps Helm chart in Argo CD...'
cat ../kubernetes/cicd/argocd/kubeapps.yml | sed "s/CLUSTER_URL/${ALPHA_CLUSTER_URL//\//\\/}/" | sed "s/ENVIRONMENT/alpha/" | kubectl apply -n argocd -f -
cat ../kubernetes/cicd/argocd/kubeapps.yml | sed "s/CLUSTER_URL/${BRAVO_CLUSTER_URL//\//\\/}/" | sed "s/ENVIRONMENT/bravo/" | kubectl apply -n argocd -f -

### Get Kubeapps endpoints & login tokens

In [ ]:
source ../config.sh

kubectl config use-context ${ALPHA_CLUSTER}

echo
echo 'Kubeapps on Alpha cluster:'
kubectl get -n kubeapps -o json service kubeapps-alpha | jq '.status.loadBalancer.ingress[0].hostname' -j

echo
echo
echo 'Alpha cluster login token:'
kubectl get -n kubeapps secret $(kubectl get -n kubeapps serviceaccount kubeapps-operator -o jsonpath='{.secrets[].name}') -o go-template='{{.data.token | base64decode}}' && echo

echo
kubectl config use-context ${BRAVO_CLUSTER}

echo
echo 'Kubeapps on Bravo cluster:'
kubectl get -n kubeapps -o json service kubeapps-bravo | jq '.status.loadBalancer.ingress[0].hostname' -j

echo
echo
echo 'Bravo cluster login token:'
kubectl get -n kubeapps secret $(kubectl get -n kubeapps serviceaccount kubeapps-operator -o jsonpath='{.secrets[].name}') -o go-template='{{.data.token | base64decode}}' && echo